In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [37]:
with open('intents.json', encoding="utf8") as content:
   data1 = json.load(content)

In [38]:
# getting the data into list format
tags = []
inputs = []
responses = {}
for intent in data1["intents"]:
   responses[intent["tag"]] = intent['responses']
   for pattern in intent["patterns"]:
      inputs.append(pattern)
      tags.append(intent['tag'])



In [39]:
for response in responses["ترحيب"]:
   print(response)
print("--------------------------------------------")
print(inputs[177:180], tags[177:180])
print(inputs[0:2], tags[0:2])

ازيك ، يارب يومك يكون كويس.ازاي اقدر اساعدك؟
اهلا، اقدر اساعد حضرتك في ايه ؟
اهلا بيك، يا تري اقدر اساعدك ازاي
عامل ايه، قولي اساعدك ازاي ؟
اقدر اساعد حضرتك ازاي؟
--------------------------------------------
['عيادة', 'عايز اعرف في كام عيادة هنا', 'اية العيادات المتسجلة عالموعع ده'] ['عيادة', 'عيادة', 'عيادة']
['مساء الخير', 'صباح الخير'] ['ترحيب', 'ترحيب']


In [40]:
# tabulaize our data 
data = pd.DataFrame({
   "inputs": inputs,
   "tags": tags
})
data

,inputs,tags
0,مساء الخير,ترحيب
1,صباح الخير,ترحيب
2,هاي,ترحيب
3,اهلا,ترحيب
4,صباحو,ترحيب
...,...,...
1394,i wanna add phone no,تعديل بروفايل
1395,i wanna change my address,تعديل بروفايل
1396,اجيب فصيلة الدم منين,تعديل بروفايل
1397,عايز احط رقم موبايلي,تعديل بروفايل


# Pre-Processing

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [43]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data["inputs"])
train = tokenizer.texts_to_sequences(data["inputs"])

# padding
x_train = pad_sequences(train)

# encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data["tags"])

In [53]:
x_train[17], y_train[17]
x_train[1395], y_train[1395] # 5  words in the pattern , their tag takes number 7

(array([   0,    0,    0,    0,    0,    0,   40,  392,  317,   73, 1390]), 7)

In [54]:
data["inputs"][1395]

'i wanna change my address'

In [55]:
input_shape = x_train.shape[1]
print(input_shape) # 11 inputs at the input layer

11


In [56]:
# define the vocab 
vocab = len(tokenizer.word_index)
print("number of unique words : ", vocab)

output_length = le.classes_.shape[0]
print("output length: ", output_length) # we have 23 outputs as we have 23 tags 

number of unique words :  1393
output length:  23
